In [5]:
%run DataSEIR.ipynb
import ray
from torch_geometric.data import Data

/home/super-potato/.local/lib/python3.9/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


{'node_ip_address': '192.168.178.33',
 'raylet_ip_address': '192.168.178.33',
 'redis_address': '192.168.178.33:27162',
 'object_store_address': '/tmp/ray/session_2021-09-27_10-16-43_747445_1842/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-09-27_10-16-43_747445_1842/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-09-27_10-16-43_747445_1842',
 'metrics_export_port': 50340,
 'node_id': 'ab4e6634a09580abf222ff6714f0036f26df8d48747ba9df90fb64ee'}

In [ ]:
class datagen():
    def __init__(self, batch_amount,p , CPU_threads, N, iterations, infection_probability, removal_probability, 
               latent_period, filename, one_hot_features, two_d_labels,save, show_progress, types, graph_type = 'ER'):
        self.batch_amount = batch_amount
        self.graph_type = graph_type
        self.p = p
        self.CPU_threads = CPU_threads
        self.N = N
        self.iterations = iterations
        self.infection_probability = infection_probability
        self.removal_probability = removal_probability
        self.latent_period = latent_period
        self.filename = filename
        self.one_hot_features = one_hot_features
        self.two_d_labels = two_d_labels
        self.show_progress = show_progress
        self.types = types
        self.features = 3 if types == 'SIR' else 4
        self.save = save
        
    def to_one_hot(self):
        for it, data in enumerate(self.dataset):
            one_hot = torch.zeros((data.x.shape[0], self.features))
            one_hot[np.arange(data.x.shape[0], dtype=int), data.x[:,0].long()]= 1
            data.x = one_hot

    def from_one_hot(self):
        prog = 0
        l = len(self.dataset)
        for it, data in enumerate(self.dataset):
            if data.y.ndim != 2:
                print("data {} does not have 2 dims... skipping".format(it))
            else:
                data.y = data.y[:,1]
        print("dataset now converted from one-hot")

    # declare wrapper for ray (multithreading for faster datageneration)
    # execute SEIRdata from DataSEIR.ipynb
    @ray.remote
    def SEIR_gen(self):
        model = SEIRdata(batches=self.batch_amount, show_progress=self.show_progress, graph_type=self.graph_type, one_hot_target=True, iterations=self.iterations, N=self.N,p = self.p, beta=self.infection_probability, gamma=self.removal_probability, alpha=self.latent_period)
        return model.generate_data()

    def generate(self):
        # generate data and distribute over available cores
        ray.init()
        xs = [self.SEIR_gen.remote(self) for i in range(self.CPU_threads)]
        xs = ray.get(xs)
        ray.shutdown()
        print('data generated.')
        # transform generated data into desired format
        self.dataset = []
        for z in xs:
            for graph_index in range(self.batch_amount):
                self.dataset.append(Data(edge_index = z[graph_index][1], x = z[graph_index][2], y = z[graph_index][3]))
        # transform features into onehot format
        if self.one_hot_features == True:
            print('one_hot')
            self.to_one_hot()
            print('onehot encoding complete.')
        else:
            pass
        if self.two_d_labels == False: self.from_one_hot()
        #save data as one file
        if not self.filename:
            self.filename = str(self.iterations)+'its_'+str(self.N)+'N_'+self.graph_type+'_type'
        else:
            pass
        torch.save(self.dataset, self.filename+".pt")
        if self.save == True: print('data saved.')
    def show(self):
        return self.dataset